# MLM Paramater Sweep


Scoring documents using the Mixture of Language Models (MLM) approach. 
This notebook's purpose is to do a parameter sweep of the field weights and the smoothing paramater λ (Lambda). 
This notebook gave a general pointer as to what smoothing parameters is good to use.

!NOTE! Runtime:  25-30 min 

In [1]:
from elasticsearch import Elasticsearch
import math
import pprint
import time

In [2]:
INDEX_NAME = "aquaint"
DOC_TYPE = "doc"

In [3]:
QUERY_FILE = "data/queries.txt"

In [4]:
OUTPUT_FILE = "data/mlm_opt.txt"  # output the ranking
QRELS_FILE = "data/qrels2.csv" # for ranking

Document fields used for scoring.

In [5]:
FIELDS = ["title", "content"]

Field weights. You'll need to set these properly in Part 3 of the assignment. For now, you can use these values.

In [6]:
FIELD_WEIGHTS = [0.1, 0.9]

Smoothing: we use Jelinek-Mercer smoothing here with the following lambda parameter. (I.e., the same smoothing parameter is used for all fields.)

In [7]:
LAMBDA = 0.1

### Load the queries from the file

See the assignment description for the format of the query file [here](https://github.com/kbalog/uis-dat630-fall2017/tree/master/assignment-1#queries).

In [8]:
def load_queries(query_file):
    queries = {}
    with open(query_file, "r") as fin:
        for line in fin.readlines():
            qid, query = line.strip().split(" ", 1)
            queries[qid] = query
    return queries

## Query analyzer

See [indices.analyze](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.analyze).

In [9]:
def analyze_query(es, query):
    tokens = es.indices.analyze(index=INDEX_NAME, body={"text": query})["tokens"]
    query_terms = []
    for t in sorted(tokens, key=lambda x: x["position"]):
        query_terms.append(t["token"])
    return query_terms

In [10]:
def grange(x, y, jump):
    while x<y:
        yield x
        x += jump

# Evaluation from Notebook 3


In [11]:
def eval_query(ranking, gt):
    p10, ap, rr, num_rel = 0, 0, 0, 0
        
    
    for i, doc_id in enumerate(ranking):
        if doc_id in gt:  # doc is relevant
            num_rel += 1  
            pi = num_rel / (i + 1)  # P@i
            ap += pi  # AP
            
            if i < 10:  # P@10
                p10 += 1
            if rr == 0:  # Reciprocal rank
                rr = 1 / (i + 1)
    p10 /= 10
    ap /= len(gt)
    
    return {"P10": p10, "AP": ap, "RR": rr}

In [12]:
def eval(gt_file, output_file):
    # load data from ground truth file
    gt = {}  # holds a list of relevant documents for each queryID
    with open(gt_file, "r") as fin:
        header = fin.readline().strip()
        if header != "queryID,docIDs":
            raise Exception("Incorrect file format!")
        for line in fin.readlines():
            qid, docids = line.strip().split(",")
            gt[qid] = docids.split()

    # load data from output file
    output = {}
    with open(output_file, "r") as fin:
        header = fin.readline().strip()
        if header != "QueryId,DocumentId":
            raise Exception("Incorrect file format!")
        for line in fin.readlines():
            qid, docid = line.strip().split(",")
            if qid not in output:
                output[qid] = []
            output[qid].append(docid)

    # evaluate each query that is in the ground truth
    #print("  QID  P@10   (M)AP  (M)RR")
    sum_p10, sum_ap, sum_rr, length = 0, 0, 0, 0
    for qid in sorted(gt.keys()):
        res = eval_query(output.get(qid, []), gt.get(qid, []))
        sum_p10 += res["P10"]
        sum_ap += res["AP"]
        sum_rr += res["RR"]
        length += 1

    # TODO compute averages over the entire query set
    sum_p10 = sum_p10/length
    sum_ap = sum_ap/length
    sum_rr = sum_rr/length
    # print averages
    #print(sum_ap)
    print("\t%5s %6.3f %6.3f %6.3f " % ("ALL", sum_p10, sum_ap, sum_rr))
    #return sum_ap

### Collection Language Model class

This class is used for obtaining collection language modeling probabilities $P(t|C_i)$.

The reason this class is needed is that `es.termvectors` does not return term statistics for terms that do not appear in the given document. This would cause problems in scoring documents that are partial matches (do not contain all query terms in all fields). 

The idea is that for each query term, we need to find a document that contains that term. Then the collection term statistics are available from that document's term vector. To make sure we find a matching document, we issue a [boolean (match)](https://www.elastic.co/guide/en/elasticsearch/reference/5.5/query-dsl-match-query.html) query.

In [13]:
class CollectionLM(object):
    def __init__(self, es, qterms):
        self._es = es
        self._probs = {}
        # computing P(t|C_i) for each field and for each query term
        for field in FIELDS:
            self._probs[field] = {}
            for t in qterms:
                self._probs[field][t] = self.__get_prob(field, t)
        
    def __get_prob(self, field, term):
        # use a boolean query to find a document that contains the term
        hits = self._es.search(index=INDEX_NAME, body={"query": {"match": {field: term}}},
                               _source=False, size=1).get("hits", {}).get("hits", {})
        doc_id = hits[0]["_id"] if len(hits) > 0 else None
        if doc_id is not None:
            # ask for global term statistics when requesting the term vector of that doc (`term_statistics=True`)
            tv = self._es.termvectors(index=INDEX_NAME, doc_type=DOC_TYPE, id=doc_id, fields=field,
                                      term_statistics=True)["term_vectors"][field]
            ttf = tv["terms"].get(term, {}).get("ttf", 0)  # total term count in the collection (in that field)
            sum_ttf = tv["field_statistics"]["sum_ttf"]
            #pp = pprint.PrettyPrinter(indent=4)
            #pp.pprint(tv)
            return ttf / sum_ttf
        
        return 0  # this only happens if none of the documents contain that term

    def prob(self, field, term):
        return self._probs.get(field, {}).get(term, 0)

### Document scorer

**TODO** This is the only method that you need to complete.

In [14]:
def score_mlm(es, clm, qterms, doc_id):
    score = 0  # log P(q|d)
    
    # Getting term frequency statistics for the given document field from Elasticsearch
    # Note that global term statistics are not needed (`term_statistics=False`)
    tv = es.termvectors(index=INDEX_NAME, doc_type=DOC_TYPE, id=doc_id, fields=FIELDS,
                              term_statistics=False).get("term_vectors", {})

    # NOTE: Keep in mind that a given document field might be empty. In that case there is no tv[field].
    #print(tv['terms'][t]['term_freq'])
    # scoring the query
    a, b, c = 0, 0, 0
    for t in qterms:
        Pt_theta_d = 0  # P(t|\theta_d)
        for i, field in enumerate(FIELDS):
            
            Pt_theta_di = 0 
            if (field in tv) and (t in tv[field]["terms"]):
                #Pt_theta_di = (1-LAMBDA)*((tv["terms"][t]['term_freq']) / (len(tv['terms']))) + (LAMBDA*((tv['field_statistics']['sum_ttf'])/(tv['field_statistics']['sum_doc_freq'])))
                # TODO compute the field language model $P(t|\theta_{d_i})$ with Jelinek-Mercer smoothing
                #tv2 = tv['terms'][t]['term_freq']
                
                Pt_theta_di = (1-LAMBDA)*((tv[field]["terms"][t]['term_freq']) / (len(tv[field]["terms"]))) + (LAMBDA* (clm._probs[field][t]))
                #a = (tv['field_statistics']['sum_ttf'])
                # b = (tv['field_statistics']['sum_doc_freq'])
                #c= a/b
                
                # NOTE keep in mind that the term vector will not contain `term` as a key if the document doesn't
                # contain that term; you will still need to use the background term probabilities for that term.
                # You can get the background term probability using `clm.prob(field, t)`
            elif(field in tv):
                Pt_theta_d= clm.prob(field,t)
            Pt_theta_d += FIELD_WEIGHTS[i] * Pt_theta_di
        
        # TODO uncomment this line once you computed Pt_theta_d (and it is >0)
        if Pt_theta_d>0:
            score += math.log(Pt_theta_d)          
        #print(Pt_theta_di)
    return score

## Main

In [15]:
es = Elasticsearch()

In [16]:
SIM = {"similarity": {"default": { "type": "BM25","b": 0.75,"k1": 1.2}}} #Ensuring Default BM25 Parameters

In [17]:
es.indices.close(index=INDEX_NAME)
es.indices.put_settings(index=INDEX_NAME, body=SIM)
es.indices.open(index=INDEX_NAME)
time.sleep(2)

In [18]:
queries = load_queries(QUERY_FILE)

In [19]:

for x in grange(0.1,1.0, 0.1) :    #lambda range
    LAMBDA = x
    print("Scoring mlm for lambda: "+str(LAMBDA))
    
    while FIELD_WEIGHTS[0] <= 0.9:
        print("\t Field Weights: ["+str(FIELD_WEIGHTS[0])+"] ["+str(FIELD_WEIGHTS[1])+"]")
        print("\t QID  P@10   (M)AP  (M)RR")
        with open(OUTPUT_FILE, "w") as fout:
            # write header
            fout.write("QueryId,DocumentId\n")
            for qid, query in queries.items():
                # get top 200 docs using BM25
                #print("Get baseline ranking for [%s] '%s'" % (qid, query))
                res = es.search(index=INDEX_NAME, q=query, df="content", _source=False, size=200).get('hits', {})
                pp = pprint.PrettyPrinter(indent=4)

                # re-score docs using MLM
                #print("Re-scoring documents using MLM")
                # get analyzed query
                qterms = analyze_query(es, query)
                #pp.pprint(qterms)
                # get collection LM 
                # (this needs to be instantiated only once per query and can be used for scoring all documents)
                clm = CollectionLM(es, qterms)        
                scores = {}
                for doc in res.get("hits", {}):
                    doc_id = doc.get("_id")
                    scores[doc_id] = score_mlm(es, clm, qterms, doc_id)

                # write top 100 results to file
                for doc_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:100]:            
                    fout.write(qid + "," + doc_id + "\n")
                fout.close
                
                
        eval(QRELS_FILE, OUTPUT_FILE)
        
        FIELD_WEIGHTS[0] +=  0.1
        FIELD_WEIGHTS[1] -=  0.1
        
    FIELD_WEIGHTS = [0.1,0.9]
   

Scoring mlm for lambda: 0.1
	 Field Weights: [0.1] [0.9]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.184  0.069  0.317 
	 Field Weights: [0.2] [0.8]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.180  0.073  0.338 
	 Field Weights: [0.30000000000000004] [0.7000000000000001]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.171  0.075  0.332 
	 Field Weights: [0.4] [0.6000000000000001]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.171  0.076  0.332 
	 Field Weights: [0.5] [0.5000000000000001]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.176  0.075  0.324 
	 Field Weights: [0.6] [0.40000000000000013]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.169  0.078  0.336 
	 Field Weights: [0.7] [0.30000000000000016]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.162  0.076  0.331 
	 Field Weights: [0.7999999999999999] [0.20000000000000015]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.167  0.075  0.326 
	 Field Weights: [0.8999999999999999] [0.10000000000000014]
	 QID  P@10   (M)AP  (M)RR
	  ALL  0.164  0.071  0.322 
Scoring mlm for lambda: 0.2
	 Field Weights: [0.1] [